In [1]:
import pandas as pd

# C Kernel Results:
1. Function: refers to the function tested in that iteration
2. Dimension: the number of dimensions in the function
3. Loop: the current iteration of testing for that function
4. Time Taken: the number of milliseconds it took to finish that iteration
5. Fitness: the best fitness found in that iteration

In [5]:
results_c_df = pd.read_csv('result_c.csv')

# Rename Function to words (0 = Sphere, 1 = Ackley, 2 = Rosenbrock) for easier recognition
func_map = {0: "Sphere", 1: "Ackley", 2: "Rosenbrock"}
results_c_df["Function"] = results_c_df["Function"].astype(int).map(func_map)

results_c_df

,Function,Dimension,Loop,Time Taken,Fitness
0,Sphere,4,0,31.889,0.000000
1,Sphere,4,1,32.744,0.000000
2,Sphere,4,2,32.297,0.000000
3,Sphere,4,3,32.593,0.000000
4,Sphere,4,4,31.550,0.000000
...,...,...,...,...,...
265,Rosenbrock,1024,25,921153.309,85.459360
266,Rosenbrock,1024,26,980926.273,81.220970
267,Rosenbrock,1024,27,990032.511,75.296377
268,Rosenbrock,1024,28,932351.978,86.868092


# CUDA Kernel Results:
1. Function: refers to the function tested in that iteration
2. Dimension: the number of dimensions in the function
3. Loop: the current iteration of testing for that function
4. Fitness: the best fitness found in that iteration

NOTE: The time taken for CUDA kernels has to be calculated manually by observing NSYS results

In [7]:
results_cuda_df = pd.read_csv('result_cuda.csv')
results_cuda_df

,Function,Dimension,Loop,Fitness
0,Sphere,4,0,0.000000
1,Sphere,4,1,0.000000
2,Sphere,4,2,0.000000
3,Sphere,4,3,0.000000
4,Sphere,4,4,0.000000
...,...,...,...,...
355,Rosenbrock,8192,25,1541.642641
356,Rosenbrock,8192,26,4194.227511
357,Rosenbrock,8192,27,2183.262175
358,Rosenbrock,8192,28,1955.522865


# Correctness comparison

This section aims to evaluate the correctness of both the CUDA and C implementations of the CPSO-S Algorithm. Since the algorithm has some inherent randomness, its solutions would sometimes be different. This would mean that each iteration might have a different solution or fitness, despite having the exact same configuration, making it hard to consistently reach the **absolutely** correct answer. Given this, we would be considering the fitness/solution of the iteration to be correct if it is lower than a certain threshold (0.25 in this case).

The configuration of the algorithm runs (CUDA and C are identical) are as follows:
- **particles** - 100
- **iterations** - 500 (iterations of a single CPSO run)
- **loops** - 30 (how many times the algorithm was ran)

Three functions were used for evaluation, alongside their minimal value and their positions.
1. Sphere: f(x) = 0 with domain x = (0, 0, ...,0)
2. Ackley: f(x) = 0 with domain x = (0, 0, 0, ..., 0)
3. Rosenbrock: f(x) = 0 with domain x = (1, 1, ..., 1)

Overall, it can be observed that both implementations had similar correctness.

It can also be observed that the CPSO-S algorithm consistently converges to a near-optimum solution when using the Ackley and Sphere functions, regardless of the increase in dimensions.

However, in using the Rosenbrock function, it could be observed that it has a hard time converging to a near-optimum solution and it worsens with more dimensions with it not having any "correct" solutions past 4-dimensions.


In [45]:
def count_correct(df, tolerance = 0.25):
  group = df.groupby(["Function", "Dimension"])["Fitness"]
  out = group.agg(count=lambda s: (s <= tolerance).sum(),
                  mean=lambda s: (s <= tolerance).mean()).reset_index()
  out = out.rename(columns={"count": "Correctness Count", "mean": "Correctness Mean"})
  out["tolerance"] = tolerance
  return out.sort_values(["Function", "Dimension"])

In [46]:
count_correct(results_c_df)

,Function,Dimension,Correctness Count,Correctness Mean,tolerance
0,Ackley,4,28,0.933333,0.25
1,Ackley,256,30,1.000000,0.25
2,Ackley,1024,30,1.000000,0.25
3,Rosenbrock,4,15,0.500000,0.25
4,Rosenbrock,256,0,0.000000,0.25
5,Rosenbrock,1024,0,0.000000,0.25
6,Sphere,4,30,1.000000,0.25
7,Sphere,256,30,1.000000,0.25
8,Sphere,1024,30,1.000000,0.25


In [ ]:
count_correct(results_cuda_df)

,Function,Dimension,Correctness Count,Correctness Mean,tolerance
0,Ackley,4,30,1.000000,0.25
1,Ackley,256,30,1.000000,0.25
2,Ackley,1024,30,1.000000,0.25
3,Ackley,8192,30,1.000000,0.25
4,Rosenbrock,4,16,0.533333,0.25
5,Rosenbrock,256,0,0.000000,0.25
6,Rosenbrock,1024,0,0.000000,0.25
7,Rosenbrock,8192,0,0.000000,0.25
8,Sphere,4,30,1.000000,0.25
9,Sphere,256,30,1.000000,0.25
